<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabiojuliodaluz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('wordcloud')\
    .getOrCreate()


In [26]:
def trata_tweets(df):
    words = df\
        .select(f.explode(f.split(f.lower('_c0'), " ")).alias('word'))\
        .withColumn('word', f.regexp_replace('word', r'http\S+', ''))\
        .withColumn('word', f.regexp_replace('word', 'rt', ''))\
        .na.replace('', None)\
        .na.drop()
    return words

In [27]:

stop = stopwords.words('portuguese')
stop.append('futebol')
plt.figure(figsize=(20, 10)) 
while True:
    try:
        words = spark.read.csv('./csv', encoding='latin1')
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + '' + row['data']
        wordcloud = WordCloud(stopwords=stop,
                                background_color='black',
                                width=1920,
                                height=100,
                                max_words=100).generate(all_words)
        plt.cla()
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gfc())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break


AnalysisException: Path does not exist: file:/Users/fabiojuliodaluz/Documents/GitHub/portfolio/dados/spark/streaming_de_dados/csv

<Figure size 2000x1000 with 0 Axes>